# Evristika

## Originalna

In [1]:
import numpy as np  # Импортирање на numpy библиотеката за работа со 2D низи.

def evaluate_state_hexapawn(board, player):
    """
    Евристичка функција за Hexapawn.
    
    Parameters:
    - board: numpy 2D array што ја претставува таблата.
    - player: 'W' или 'B' за бел или црн играч.
    
    Returns:
    - Вредност на евристиката за дадената состојба.
    """
    n, m = board.shape  # Добиј ги димензиите на таблата.
    opponent = 'W' if player == 'B' else 'B'  # Определи го противникот.

    # 1. **Прогрес на пионите:**
    progress_score = 0
    for x in range(n):
        for y in range(m):
            if board[x, y] == player:  # Ако полето има пион на тековниот играч,
                progress_score += (n - 1 - x) if player == 'W' else x  # додади поени за напредок.
            elif board[x, y] == opponent:  # Ако полето има пион на противникот,
                progress_score -= (n - 1 - x) if opponent == 'W' else x  # одземи поени за напредок на противникот.

    # 2. **Мобилност:**
    def pawn_moves(px, py, pl):
        """Генерирај сите можни потези за даден пион."""
        moves = []
        direction = -1 if pl == 'W' else 1  # Одреди го правецот на движење.
        # Напред
        if 0 <= px + direction < n and board[px + direction, py] == '.':
            moves.append((px + direction, py))  # Додади го напредниот потез.
        # Дијагонални потези (фаќање противник)
        for dy in [-1, 1]:
            nx, ny = px + direction, py + dy
            if 0 <= nx < n and 0 <= ny < m and board[nx, ny] == opponent:
                moves.append((nx, ny))  # Додади ги дијагоналните потези (фаќање противник).
        return moves

    mobility_score = 0
    for x in range(n):
        for y in range(m):
            if board[x, y] == player:  # Ако полето има пион на тековниот играч,
                mobility_score += len(pawn_moves(x, y, player))  # додади поени за можни потези.
            elif board[x, y] == opponent:  # Ако полето има пион на противникот,
                mobility_score -= len(pawn_moves(x, y, opponent))  # одземи поени за можните потези на противникот.

    # 3. **Закани:**
    threat_score = 0
    for x in range(n):
        for y in range(m):
            if board[x, y] == player:  # Ако полето има пион на тековниот играч,
                for move in pawn_moves(x, y, player):
                    if board[move[0], move[1]] == opponent:  # Ако можниот потез води до фаќање противник,
                        threat_score += 1  # додади поени за фаќање противник.
            elif board[x, y] == opponent:  # Ако полето има пион на противникот,
                for move in pawn_moves(x, y, opponent):
                    if board[move[0], move[1]] == player:  # Ако можниот потез води до фаќање наш пион,
                        threat_score -= 1  # одземи поени за фаќање на наш пион.

    # Вкупна евристичка вредност
    w1, w2, w3 = 3.0, 2.0, 4.0  # Тежини за секоја компонента.
    total_heuristic = w1 * progress_score + w2 * mobility_score + w3 * threat_score  # Финална евристика.
    
    return total_heuristic  # Врати ја финалната евристичка вредност.

In [2]:
board = np.array([
    ['.', 'B', '.'],
    ['.', '.', 'W'],
    ['W', '.', '.']
])

player = 'W'
heuristic_value = evaluate_state_hexapawn(board, player)
print(f"Евристичката вредност за играчот {player} е: {heuristic_value}")

Евристичката вредност за играчот W е: 11.0


## Izmeneta

In [15]:
import numpy as np  # Import numpy for working with 2D arrays.

def evaluate_hexapawn_state_debug(board, player):
    """
    Debugged heuristic evaluation function for Hexapawn.

    Parameters:
    - board: numpy 2D array representing the game board.
    - player: 'W' or 'B' for the current player (White or Black).

    Returns:
    - Heuristic value of the given board state for the player.
    """
    rows, cols = board.shape  # Get board dimensions.
    opponent = 'W' if player == 'B' else 'B'  # Determine opponent.

    # Helper function to compute pawn movement options.
    def get_moves(x, y, pawn_player):
        direction = -1 if pawn_player == 'W' else 1  # Movement direction.
        moves = []

        # Forward move
        if 0 <= x + direction < rows and board[x + direction, y] == '.':
            moves.append((x + direction, y))

        # Diagonal captures
        for dy in [-1, 1]:
            nx, ny = x + direction, y + dy
            if 0 <= nx < rows and 0 <= ny < cols and board[nx, ny] == opponent:
                moves.append((nx, ny))

        return moves

    progress_score = 0
    mobility_score = 0
    threat_score = 0

    # Evaluate board positions
    for x in range(rows):
        for y in range(cols):
            if board[x, y] == '.':
                continue

            current_pawn = board[x, y]
            is_player_pawn = (current_pawn == player)
            is_opponent_pawn = (current_pawn == opponent)

            # Calculate progress score
            if is_player_pawn:
                progress_score += (rows - 1 - x) if player == 'W' else x
            elif is_opponent_pawn:
                progress_score -= (rows - 1 - x) if opponent == 'W' else x

            # Calculate mobility score
            moves = get_moves(x, y, current_pawn)
            if is_player_pawn:
                mobility_score += len(moves)
            elif is_opponent_pawn:
                mobility_score -= len(moves)

            # Calculate threat score
            for move in moves:
                target_x, target_y = move
                if board[target_x, target_y] == opponent and is_player_pawn:
                    threat_score += 1
                elif board[target_x, target_y] == player and is_opponent_pawn:
                    threat_score -= 1

    # Combine scores using weights
    weights = {'progress': 3.0, 'mobility': 2.0, 'threat': 4.0}
    heuristic_value = (
        weights['progress'] * progress_score +
        weights['mobility'] * mobility_score +
        weights['threat'] * threat_score
    )

    # Return detailed debug information and the result
    return {
        
        "heuristic_value": heuristic_value
    }

# Example input
board = np.array([
    ['.', 'B', '.'],
    ['.', '.', 'W'],
    ['W', '.', '.']
])

player = 'W'
heuristic_value = evaluate_hexapawn_state_debug(board, player)
print(f"Heuristic details: {heuristic_value}")

Heuristic details: {'heuristic_value': 11.0}
